### LLMs

In [7]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI()
chat_model.predict("hi!")

'Hello! How can I assist you today?'

In [17]:
from langchain.schema import HumanMessage

prompt = "What would be a good company name for a company that makes maps?"
messages = [HumanMessage(content=prompt)]
chat_model.predict_messages(messages)

AIMessage(content='GeoCartography')

### Prompt templates


In [19]:
from langchain.prompts import PromptTemplate

# prompt template allow to parameterize the prompt
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
print(prompt.format(product="colorful socks"))

# chat prompt templates allow to also parametrize the role, position in the list, etc.
from langchain.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

What is a good name for a company that makes colorful socks?


[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

### Output parsers

In [20]:
# Convert the raw output of an LLM into a format that can be used downstream
#  1. Convert text from LLM -> structured information (e.g. JSON)
#  2. Convert a ChatMessage into just a string
#  3. Convert the extra information returned from a call besides the message (like OpenAI function invocation) into a string.

from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")

['hi', 'bye']

### Combine into 1 chain: LLM + prompt template + output parser

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

# Note that we are using the | syntax to join these components together. 
# This | syntax is called the LangChain Expression Language.
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()
chain.invoke({"text": "countries"})

['Canada', 'United States', 'United Kingdom', 'Australia', 'Germany']